### 作業目的: 熟練以Torchtext進行文本資料讀取

本次作業主要會使用[polarity](http://www.cs.cornell.edu/people/pabo/movie-review-data/)的電影評論來進行使用torchtext資料讀取，學員可以在附件的polarity.tsv看到所使用的資料。

Hint: 這次作業同學可以嘗試使用[torchtext.data.TabularDataset](https://torchtext.readthedocs.io/en/latest/data.html#tabulardataset)，可以更簡易讀取資料

### 載入套件

In [1]:
import torch
import pandas as pd
import numpy as np
from torchtext import data, datasets

In [2]:
# 探索資料
# 可以發現資料為文本與類別，而類別即為正評與負評
input_data = pd.read_csv('./polarity.tsv', delimiter='\t', header=None, names=['text', 'label'])
input_data

,text,label
0,films adapted from comic books have had plenty...,1
1,every now and then a movie comes along from a ...,1
2,you've got mail works alot better than it dese...,1
3,jaws is a rare film that grabs your attentio...,1
4,moviemaking is a lot like being the general ma...,1
...,...,...
1995,"if anything , "" stigmata "" should be taken as ...",0
1996,"john boorman's "" zardoz "" is a goofy cinematic...",0
1997,the kids in the hall are an acquired taste .it...,0
1998,there was a time when john carpenter was a gre...,0


### 建立Pipeline生成資料

In [4]:
import spacy

nlp = spacy.load("en_core_web_sm")

# 建立Field與Dataset
def remove_non_char(x):
    x = ' '.join(x)
    x = re.sub("[^a-zA-Z]", ' ', x)
    x = x.split()
    
    return x

text_field = data.Field(sequential=True, dtype=torch.float64, lower=True, tokenize='spacy', preprocessing=remove_non_char)
label_field = data.Field(sequential=False)

examples = []

for i in range(len(input_data)):
    examples.append(data.Example.fromlist(data = [input_data[i]['text'], input_data[i]['label']],
                                          fields = [('text', text_field), ('label', label_field)]))
    



OSError: [E941] Can't find model 'en'. It looks like you're trying to load a model from a shortcut, which is deprecated as of spaCy v3.0. To load the model, use its full name instead:

nlp = spacy.load("en_core_web_sm")

For more details on the available models, see the models directory: https://spacy.io/models. If you want to create a blank model, use spacy.blank: nlp = spacy.blank("en")

In [ ]:
# 取的examples並打亂順序
### <your code> ###

# 以8:2的比例切分examples
train_ex = examples[:int(len(examples)*0.8)]
test_ex = examples[int(len(examples)*0.8):]

# 建立training與testing dataset
train_data = data.Dataset(examples=train_ex, fields={'text': text_field, 'label' :label_field})
test_data = data.Dataset(examples=test_ex, fields={'text':text_field, 'label':label_field})

train_data[0].label, train_data[0].text

In [ ]:
# 建立字典
text_field.build_vocab(train_data)
label_field.build_vocab(train_data)

print(f"Vocabularies of index 0-5: {text_field.vocab.itos[:10]} \n")
print(f"words to index {text_field.vocab.stoi}")

In [ ]:
# create iterator for training and testing data

def get_train_test_iter(train_data, test_data):
    iter_for_train = data.Iterator(dataset=train_data, batch_size = 5, repeat=False, sort_key=lambda ex:len(ex.text))
    iter_for_test = data.Iterator(dataset=test_data, batch_size = 5, repeat=False, sort_key=lambda ex:len(ex.text))
    
    return iter_for_train, iter_for_test

train_iter, test_iter = get_train_test_iter(train_data, test_data)

In [ ]:
for train_batch in train_iter:
    print(train_batch.text, train_batch.text.shape)
    print(train_batch.label, train_batch.label.shape)
    break